In [10]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.insert(0, module_path)
import random

In [ ]:
from collections import namedtuple

GeoExtent = namedtuple('GeoExtent', ['x_min', 'y_max', 'x_max', 'y_min'])
GeoAffine = namedtuple('GeoAffine', ['ul_x', 'x_res', 'rot_1', 'ul_y', 'rot_2', 'y_res'])

CONUS_EXTENT = GeoExtent(x_min=-2565585,
                         y_min=14805,
                         x_max=2384415,
                         y_max=3314805)


def extent_from_hv(h, v, loc='conus'):
    """
    Retrieve the geospatial extent for the given tile h/v
    :param h:
    :param v:
    :param loc:
    :return:
    """
    loc = loc.lower()

    if loc == 'conus':
        xmin = CONUS_EXTENT.x_min + h * 5000 * 30
        xmax = CONUS_EXTENT.x_min + h * 5000 * 30 + 5000 * 30
        ymax = CONUS_EXTENT.y_max - v * 5000 * 30
        ymin = CONUS_EXTENT.y_max - v * 5000 * 30 - 5000 * 30
    else:
        raise Exception('Location not implemented: {0}'
                        .format(loc))

    return (GeoExtent(x_min=xmin, x_max=xmax, y_max=ymax, y_min=ymin),
            GeoAffine(ul_x=xmin, x_res=30, rot_1=0, ul_y=ymax, rot_2=0, y_res=-30))

In [13]:
import urllib2

__HOST__ = r'http://lcmap-test.cr.usgs.gov/changes/results'
__ALGORITHM__ = r'lcmap-pyccd:1.1.0'


def api_request(x, y, refresh=False):
    endpoint = '/'.join([__HOST__, __ALGORITHM__, str(x), str(y)]) + '?refresh={}'.format(str(refresh).lower())
    
    request = urllib2.Request(endpoint)

    #     base64string = (base64
    #                     .encodestring('%s:%s' % (username, password))
    #                     .replace('\n', ''))
    #     request.add_header("Authorization", "Basic %s" % base64string)

    try:
        result = urllib2.urlopen(request)
    except urllib2.HTTPError as e:
        result = e

    return json.loads(result.read())

In [16]:
def request_tile(h, v, refresh=False):
    ext, _ = extent_from_hv(h, v)
    
    resps = []
    for y in xrange(ext.y_max, ext.y_max - 5000 * 30, -3000):
        for x in xrange(ext.x_min, ext.x_min + 5000 * 30, 3000):
            resps.append(api_request(x, y, refresh=refresh))

In [15]:
request_tile(2, 9, True)

(1964805, -2265585)
